In [1]:
import subprocess
import time
import PrometheusMetricsCSV as prom
import pandas as pd
import numpy as np

In [2]:
class SwarmEnvironment:
    
    def __init__(self, numberOfTopics):
        subprocess.call("cd SwarmScripts && docker-compose up -d", shell=True)

        time.sleep(10)

        self.minNumberOfBrokers = 3
        self.maxNumberOfBrokers = 11#mexri 29 kanonika

        self.activeBrokers = 3

        self.BROKER = 3
        self.PLISTENER = 19095
        self.ALISTENER = 9095
        self.KAFKA_ID = 4
        self.AGENT = 7074

        self.producerLatency = 0 
        self.consumerLatency = 0

        self.SLA = 300

        self.n = numberOfTopics
        self.numberOfTopics = self.n
        self.numberOfProducers = self.n
        self.numberOfConsumers = self.n

        self.pw = 'thanos '
        self.remotepc = 'thanos@192.168.1.117 '
        #Running on the machine specified by the parameters pw,remotepc
        for i in range(self.numberOfTopics):
            command = 'sshpass '+'-p '+self.pw+'ssh '+self.remotepc+'bash '+'-s '+' < '+"SwarmScripts/CreateTopics.sh "+str(i)
            print(command)
            subprocess.call(command,shell=True)
            time.sleep(1)
            
        for i in range(self.numberOfConsumers):
            command = 'sshpass '+'-p '+self.pw+'ssh '+self.remotepc+'bash '+'-s '+' < '+"SwarmScripts/Consumers.sh "+str(i)
            print(command)
            subprocess.Popen(command,shell=True)
            time.sleep(1)

        for i in range(self.numberOfProducers):
            command = 'sshpass '+'-p '+self.pw+'ssh '+self.remotepc+'bash '+'-s '+' < '+"SwarmScripts/Producers.sh "+str(i)
            print(command)
            subprocess.Popen(command,shell=True)
            time.sleep(1)
            

    def state(self):
        while True:
            prom.writeMetricsCSV()
            df = pd.read_csv("prometheus.csv", header=0)
            na = np.array(df['value'])
            if (na.shape)[0] == 37:
                break
        producer = df[(df['name'] == 'quantile_total_time_per_sec:quantile_rate1m') & (df['job'] == 'Produce')]
        consumer = df[(df['name'] == 'quantile_total_time_per_sec:quantile_rate1m') & (df['job'] == 'FetchConsumer')]
        self.producerLatency = producer.iloc[0]['value']
        self.consumerLatency = consumer.iloc[0]['value']
        return np.round(na, 4)
    
    def reward(self, producerLatency, consumerLatency, function):
        if function == 0:
            
            return (-1 * self.activeBrokers)
        
        elif function == 1:
            temp = (producerLatency+consumerLatency)
            temp = ((temp-self.SLA)/self.SLA) *100
            print(f"im here:{self.producerLatency},{self.consumerLatency},{temp}")
            cost = -1 * 10
            slaCost = 0
            if temp >= 200:
                slaCost = 2*cost
            elif temp >= 150:
                slaCost = (150/100)*cost
            elif temp >= 100:
                slaCost = cost
            elif temp >= 50:
                slaCost = (25/100)*cost
            elif temp > 0:
                slaCost = (10/100)*cost
            else :
                slaCost = 0

            return slaCost
        else:
            
            normalized =  -1*(self.activeBrokers - self.minNumberOfBrokers)/(self.maxNumberOfBrokers-self.minNumberOfBrokers)
            temp = (producerLatency+consumerLatency)
            temp = ((temp-self.SLA)/self.SLA) *100
            print(f"im here:{self.producerLatency},{self.consumerLatency},{temp}")
            slaCost = -1

            if temp >= 200:
                slaCost = slaCost
            elif temp >= 150:
                slaCost = (80/100)*slaCost
            elif temp >= 100:
                slaCost = (60/100)* slaCost
            elif temp >= 50:
                slaCost = (40/100)*slaCost
            elif temp > 0:
                slaCost = (20/100)*slaCost 
            else :
                slaCost = 0

            return np.round((30/100)*normalized + (70/100)*slaCost, 4)
    
    def step(self, action, function):
        #We have 3 possible action
        #Scale up
        if action == 0:
            if self.activeBrokers >= self.maxNumberOfBrokers:
                print("No scaling up. We have reached the maximum number of brokers!")
                time.sleep(20)
                state = self.state()
                reward = self.reward(self.producerLatency,self.consumerLatency,function)
                return state, reward
            print("We are scaling up!")
            subprocess.call(["bash", "SwarmScripts/ScaleUp.sh", str(self.BROKER), str(self.PLISTENER), str(self.ALISTENER), str(self.KAFKA_ID), str(self.AGENT)])
            time.sleep(5)
            subprocess.call(["bash", "SwarmScripts/Partitioner.sh", "ScaleUp"])
            self.BROKER += 1
            self.PLISTENER += 1
            self.ALISTENER += 1
            self.KAFKA_ID += 1
            self.AGENT += 1

            self.activeBrokers += 1
        #Scale down
        elif action == 1:
            if self.activeBrokers <= self.minNumberOfBrokers:
                print("No scaling down. The minimun number of brokers must be 3!")
                time.sleep(20)
                state = self.state()
                reward = self.reward(self.producerLatency,self.consumerLatency,function)
                return state, reward
            print("we are scaling down!")
            subprocess.call(["bash", "SwarmScripts/Partitioner.sh", "ScaleDown", str(self.KAFKA_ID - 1)])
            subprocess.call(["bash", "SwarmScripts/ControlledShutdown.sh", str(self.BROKER - 1)])
            self.BROKER -= 1
            self.PLISTENER -= 1
            self.ALISTENER -= 1
            self.KAFKA_ID -= 1
            self.AGENT -= 1

            self.activeBrokers -= 1
        #Do nothing
        else:
            print(f"No scaling action was taken!")
            
        time.sleep(20)
        state = self.state()
        reward = self.reward(self.producerLatency,self.consumerLatency,function)
        return state, reward
    
    def reset(self):
        subprocess.call(["bash", "SwarmScripts/Restart.sh"])
        command = 'sshpass '+'-p '+self.pw+'ssh '+self.remotepc+'bash '+'-s '+' < '+"SwarmScripts/Restart.sh"
        subprocess.call(command,shell=True)
        subprocess.call(["bash", "SwarmScripts/DeleteNetwork.sh"])

In [3]:
# env = SwarmEnvironment(4)

In [4]:
#  env.reset()